In [3]:
# cleaning data
# The file included declines, walkaways, and booked accounts
# keep: Flags = funded or repayment
import pandas as pd
data = pd.read_csv(".\PRM.EDTOUT.DGMMOTOL.CUSTCVL2  Credit Auto.csv")
data = data.loc[(data.loc[:, 'FLAGS'] == 'funded') | (data.loc[:, 'FLAGS'] == 'repayment'), :]
data.to_csv("Observations.csv", index=False)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (127,128,129,1022,1023,1212,1214) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# read clean data
import pandas as pd
import numpy as np
# seperate data into two buckets depends on states of performance
good = ['1-30 DPD', 'Current', 'Paid', 'Matured']
bad = ['Balance Owed', 'Assigned for Repossession', 'Recovered', '90+ DPD', 'Bankruptcy']
for item in good:
    data.loc[data.loc[:, 'PERFORMANCE'] == item, 'default'] = 0
for item in bad:
    data.loc[data.loc[:, 'PERFORMANCE'] == item, 'default'] = 1  
data = data.loc[((data.loc[:, 'default'] == 0) | (data.loc[:, 'default'] == 1)), :]

# delete column if over 90% are the same or nan
row_num = data.shape[0]
for col_name in data.columns.tolist():
    non_nan = data.loc[:, col_name].count()
    if non_nan < row_num * 0.9:
        data = data.drop(col_name, 1)
# delete non meaningful column( for example: "analyticsmatchkey")
data = data.drop('analyticsmatchkey', 1)

print("Total size after cleaning ", data.shape)
print("good size = ", data.loc[data.loc[:, 'default'] == 0, :].shape)
print("bad size = ", data.loc[data.loc[:, 'default'] == 1, :].shape)

Total size after cleaning  (4972, 1070)
good size =  (3167, 1070)
bad size =  (1805, 1070)


In [5]:
# delete all column with string data type
for col_name in data.columns.tolist():
    if data.loc[:, col_name].dtype == 'object':
        data = data.drop(col_name, 1)
# clean all row with nan value
data.dropna(inplace=True)

In [6]:
# seperate for training data and testing data
train_data = data.sample(frac=0.7)
train_data.sort_index(inplace=True)
test_data = data[~data.index.isin(train_data.index)]
print("Number of training data = ", len(train_data))
print("Nuber of testing data = ", len(test_data))
train_data.to_csv('training_data.csv', index=False)
test_data.to_csv('testing_data.csv', index=False)

Number of training data =  3177
Nuber of testing data =  1362
